# Corporate Hybrid Forecast Notebook (Prophet + ARIMA) – v2


## 01 - Imports & Settings

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd

from prophet import Prophet
import pmdarima as pm
from sklearn.metrics import mean_absolute_percentage_error
from typing import Optional, List, Dict

# Pandas engines
PD_READ_XLSX_ENGINE = 'openpyxl'
PD_WRITE_XLSX_ENGINE = 'openpyxl'

# === Project paths (adapt to your local if needed) ===
INPUT_FOLDER  = r"C:\Users\pt3canro\Desktop\CAPACITY\input_model"
OUTPUT_FOLDER = r"C:\Users\pt3canro\Desktop\CAPACITY\outputs"

# Source files (as in your original pipeline)
INCOMING_PATH   = os.path.join(INPUT_FOLDER, "Incoming_new.xlsx")
CALL_PATH       = os.path.join(INPUT_FOLDER, "call_performance.xlsx")
AGENTS_PATH     = os.path.join(INPUT_FOLDER, "agent_language_n_target.xlsx")
PROD_PATH       = os.path.join(INPUT_FOLDER, "productivity_agents.xlsx")
EINSTEIN_PATH   = os.path.join(INPUT_FOLDER, "einstein.xlsx")
INVENTORY_PATH  = os.path.join(INPUT_FOLDER, "inventory_month.xlsx")
DEPT_PATH  = os.path.join(INPUT_FOLDER, "department.xlsx")   # official mapping source
DEPT_SHEET = None

# Output files
OUT_XLSX = "capacity_forecast_hybrid.xlsx"
OUT_MAPE = "mape_by_department.xlsx"

# Forecast horizon
HORIZON_MONTHS = 6

# Language shares (fixed)
LANG_SHARE = {
    'English': 64.35,
    'French' : 7.41,
    'German' : 8.60,
    'Italian': 6.67,
    'Portuguese': 1.62,
    'Spanish': 11.35,
}

## 02 - Helper functions

In [2]:
def find_first(df: pd.DataFrame, candidates: List[str]) -> Optional[str]:
    """Return the first column in df that matches any candidate (case-insensitive, normalized)."""
    low = {c.lower(): c for c in df.columns}
    for cand in candidates:
        if cand.lower() in low:
            return low[cand.lower()]
    for c in df.columns:
        cl = c.lower().replace(' ', '').replace('_', '')
        for cand in candidates:
            if cand.lower().replace(' ', '').replace('_', '') == cl:
                return c
    return None

def mape_safe(y_true: pd.Series, y_pred: pd.Series) -> Optional[float]:
    """Safe MAPE (%) ignoring zeros in denominator."""
    y_true = y_true.astype(float)
    y_pred = y_pred.astype(float)
    mask = y_true > 0
    if mask.sum() == 0:
        return None
    return float(mean_absolute_percentage_error(y_true[mask], y_pred[mask]) * 100.0)

def month_start(dt: pd.Timestamp) -> pd.Timestamp:
    return pd.Timestamp(dt.year, dt.month, 1)

# === Calendar regressors for monthly data ===
def month_calendar_features(months: pd.Series) -> pd.DataFrame:
    """Build monthly exogenous features: business days and weekend days for each month."""
    uniq = pd.Series(pd.to_datetime(months).values.astype('datetime64[M]')).drop_duplicates().sort_values()
    rows = []
    for m in uniq:
        m = pd.Timestamp(m)
        rng = pd.date_range(m, m + pd.offsets.MonthEnd(0), freq='D')
        biz = int(np.sum(rng.weekday < 5))
        wknd = int(len(rng) - biz)
        rows.append({'Month': m, 'biz_days': biz, 'weekend_days': wknd})
    return pd.DataFrame(rows)

def build_exog_for_months(month_series: pd.Series) -> pd.DataFrame:
    """Wrapper to obtain the exogenous dataframe for all relevant months (history + future)."""
    return month_calendar_features(month_series)

# === Rolling-origin backtest for Prophet/ARIMA (1-step ahead) ===
def rolling_mape_one_dep(dep_df: pd.DataFrame, model: str,
                         exog_cols=('biz_days','weekend_days'),
                         min_train=12, horizon=1) -> float:
    """
    dep_df columns: ['Month','tickets'] + exog_cols
    model: 'prophet' or 'arima'
    Returns average MAPE (%) over rolling windows. NaN if not enough data.
    """
    df = dep_df.sort_values('Month').copy()
    df['y'] = df['tickets'].astype(float)
    df['ds'] = df['Month']

    n = len(df)
    if n < (min_train + horizon):
        return np.nan

    errors = []
    for end in range(min_train, n - horizon + 1):
        train = df.iloc[:end]
        test  = df.iloc[end:end+horizon]
        if model == 'prophet':
            m = Prophet(interval_width=0.90, yearly_seasonality=False, seasonality_mode='multiplicative',
                        changepoint_prior_scale=0.2)  # slightly elastic
            for c in exog_cols:
                m.add_regressor(c)
            m.fit(train[['ds','y'] + list(exog_cols)])
            future = test[['ds']].copy()
            future = future.merge(test[['ds'] + list(exog_cols)], on='ds', how='left')
            pred = m.predict(future)['yhat'].values
        else:
            X_train = train[list(exog_cols)].values
            X_test  = test[list(exog_cols)].values
            mdl = pm.auto_arima(train['y'].values, X=X_train, seasonal=False,
                                error_action='ignore', suppress_warnings=True,
                                n_jobs=1, stepwise=True, max_p=5, max_q=5, max_d=2)
            pred = mdl.predict(horizon, X=X_test)

        y_true = test['y'].values
        mask = y_true > 0
        if mask.any():
            errors.append(mean_absolute_percentage_error(y_true[mask], pred[mask]) * 100.0)

    return float(np.mean(errors)) if errors else np.nan

# === Forecast functions with exogenous regressors ===
def forecast_prophet_exog(train_m: pd.DataFrame, exog: pd.DataFrame,
                          dep_col="Department", date_col="Month", y_col="tickets",
                          horizon=6, for_verticals=None, dep_to_vertical=None) -> pd.DataFrame:
    """Prophet with calendar regressors; tunable CPS for target verticals."""
    rows = []
    for dep, g in train_m.groupby(dep_col):
        g = g[[date_col, y_col]].dropna().sort_values(date_col)
        g = g.rename(columns={date_col:'ds', y_col:'y'})
        g = g.merge(exog.rename(columns={'Month':'ds'}), on='ds', how='left')

        # More elastic CPS for the target verticals only
        is_target = (for_verticals is not None and dep_to_vertical is not None
                     and dep_to_vertical.get(dep) in for_verticals)
        cps = 0.20 if is_target else 0.05

        try:
            m = Prophet(interval_width=0.90, yearly_seasonality=False, seasonality_mode='multiplicative',
                        changepoint_prior_scale=cps)
            for c in ('biz_days','weekend_days'):
                m.add_regressor(c)
            m.fit(g[['ds','y','biz_days','weekend_days']])

            start_fc = month_start(pd.Timestamp.today())
            future = pd.date_range(start_fc, periods=horizon, freq='MS').to_frame(index=False, name='ds')
            future = future.merge(exog.rename(columns={'Month':'ds'}), on='ds', how='left')
            pred = m.predict(future).tail(horizon)['yhat']
            fc_vals = np.maximum(0, np.round(pred)).astype(int).tolist()
        except Exception:
            fc_vals = [int(g['y'].iloc[-1])] * horizon if len(g) else [0] * horizon

        for d, v in zip(pd.to_datetime(future['ds']), fc_vals):
            rows.append({dep_col: dep, date_col: d, 'Forecast_Prophet': int(v)})

    return pd.DataFrame(rows)

def forecast_arima_exog(train_m: pd.DataFrame, exog: pd.DataFrame,
                        dep_col="Department", date_col="Month", y_col="tickets",
                        horizon=6, for_verticals=None, dep_to_vertical=None) -> pd.DataFrame:
    """Auto-ARIMA with exogenous regressors; wider search for target verticals."""
    rows = []
    for dep, g in train_m.groupby(dep_col):
        g = g[[date_col, y_col]].dropna().sort_values(date_col)
        X = exog.merge(g[[date_col]], left_on='Month', right_on=date_col, how='right')[['biz_days','weekend_days']].values
        y = g[y_col].astype(float).values

        if len(y) >= 6:
            try:
                is_target = (for_verticals is not None and dep_to_vertical is not None
                             and dep_to_vertical.get(dep) in for_verticals)
                stepwise = not is_target  # allow broader search only for target verticals
                mdl = pm.auto_arima(y, X=X, seasonal=False, error_action='ignore', suppress_warnings=True,
                                    stepwise=stepwise, max_p=5, max_q=5, max_d=2)
                start_fc = month_start(pd.Timestamp.today())
                future = pd.date_range(start_fc, periods=horizon, freq='MS')
                Xf = exog[exog['Month'].isin(future)][['biz_days','weekend_days']].values
                fc = mdl.predict(horizon, X=Xf)
                fc_vals = np.maximum(0, np.round(fc)).astype(int).tolist()
            except Exception:
                fc_vals = [int(y[-1])] * horizon
        else:
            fc_vals = [int(y[-1])] * horizon if len(y) else [0] * horizon

        for d, v in zip(pd.date_range(month_start(pd.Timestamp.today()), periods=horizon, freq='MS'), fc_vals):
            rows.append({dep_col: dep, date_col: d, 'Forecast_ARIMA': int(v)})

    return pd.DataFrame(rows)

# === Hybrid selection with CV (realistic accuracy) ===
def build_hybrid_table_cv(train_m: pd.DataFrame, exog: pd.DataFrame,
                          dep_col='Department', date_col='Month', y_col='tickets',
                          for_verticals=None, dep_to_vertical=None, min_train=12) -> pd.DataFrame:
    out = []
    t = train_m.merge(exog, on=date_col, how='left')
    for dep, g in t.groupby(dep_col):
        m_p = rolling_mape_one_dep(g[[date_col, y_col, 'biz_days','weekend_days']], 'prophet', min_train=min_train)
        m_a = rolling_mape_one_dep(g[[date_col, y_col, 'biz_days','weekend_days']], 'arima',   min_train=min_train)
        if np.isnan(m_p) and np.isnan(m_a):
            best = 'Prophet'
        elif np.isnan(m_p):
            best = 'ARIMA'
        elif np.isnan(m_a):
            best = 'Prophet'
        else:
            best = 'Prophet' if m_p <= m_a else 'ARIMA'
        out.append({dep_col: dep, 'MAPE_Prophet_CV': m_p, 'MAPE_ARIMA_CV': m_a, 'Best_Model': best})
    return pd.DataFrame(out)

def apply_hybrid(kpi: pd.DataFrame, table: pd.DataFrame, dep_col='Department') -> pd.DataFrame:
    kpi = kpi.copy()
    kpi['Forecast_Hybrid'] = 0
    best_map = table.set_index(dep_col)['Best_Model'].to_dict()
    for dep, best in best_map.items():
        src = 'Forecast_Prophet' if best == 'Prophet' else 'Forecast_ARIMA'
        kpi.loc[kpi[dep_col] == dep, 'Forecast_Hybrid'] = kpi.loc[kpi[dep_col] == dep, src].values
    return kpi

# === Language split (fixed shares) ===
def language_split(df_fc: pd.DataFrame, dep_col='Department', date_col='Month',
                   fc_col='Forecast_Hybrid', lang_share: Dict[str, float] = LANG_SHARE) -> pd.DataFrame:
    rows = []
    shares = {k: float(v)/100.0 for k, v in lang_share.items()}
    for _, r in df_fc[[dep_col, date_col, fc_col]].iterrows():
        for lang, s in shares.items():
            rows.append({
                dep_col: r[dep_col],
                date_col: r[date_col],
                'Language': lang,
                'Forecast_Hybrid_Lang': int(round(r[fc_col]*s))
            })
    return pd.DataFrame(rows)

## 03 - Load Data

In [3]:
# === Load input dataframes (patched for department.xlsx) ===
incoming     = pd.read_excel(INCOMING_PATH,  engine=PD_READ_XLSX_ENGINE)
call         = pd.read_excel(CALL_PATH,      engine=PD_READ_XLSX_ENGINE)
agents       = pd.read_excel(AGENTS_PATH,    engine=PD_READ_XLSX_ENGINE)
productivity = pd.read_excel(PROD_PATH,      engine=PD_READ_XLSX_ENGINE)
einstein     = pd.read_excel(EINSTEIN_PATH,  engine=PD_READ_XLSX_ENGINE)
inventory    = pd.read_excel(INVENTORY_PATH, engine=PD_READ_XLSX_ENGINE)

# Official department mapping (primary source)
try:
    dept_sheets = pd.read_excel(DEPT_PATH, engine=PD_READ_XLSX_ENGINE, sheet_name=None)  # may return dict
    # Pick preferred sheet if present; otherwise first sheet
    preferred = "CustomerService department"
    if isinstance(dept_sheets, dict):
        if preferred in dept_sheets:
            dept_map_df = dept_sheets[preferred]
        else:
            # take the first sheet
            first_name = next(iter(dept_sheets.keys()))
            dept_map_df = dept_sheets[first_name]
    else:
        dept_map_df = dept_sheets  # already a DataFrame
except Exception as e:
    print(f"[WARN] Could not read department.xlsx: {e}")
    dept_map_df = pd.DataFrame()

print('Files loaded:',
      len(incoming), len(call), len(agents), len(productivity), len(einstein), len(inventory),
      "(+ dept map rows:", len(dept_map_df), ")")

Files loaded: 13689 38177 2402 26508 440578 76 (+ dept map rows: 109 )


## 04 - Preprocessing – Build train_m (Department, Month, tickets)

In [4]:
# === Preprocessing — Build monthly training table (Department, Month, tickets) ===
cand_date = ['Date', 'date', 'Created Date', 'Ticket Date']
cand_dep  = ['Department', 'department', 'Department Name', 'dept_name', 'department_id']
cand_val  = ['total_incoming', 'ticket_total', 'volume', 'count', 'Tickets', 'tickets']

col_date = find_first(incoming, cand_date)
col_dep  = find_first(incoming, cand_dep)
col_val  = find_first(incoming, cand_val)

if col_date is None or col_dep is None or col_val is None:
    raise ValueError(f"Incoming file must contain date/department/tickets-like columns. Found: {col_date}, {col_dep}, {col_val}")

inc = incoming[[col_date, col_dep, col_val]].copy()
inc.columns = ['Date', 'Department', 'tickets']
inc['Date']  = pd.to_datetime(inc['Date'], errors='coerce')
inc['Month'] = inc['Date'].values.astype('datetime64[M]')

train_m = (
    inc.groupby(['Department', 'Month'], as_index=False)['tickets']
      .sum()
      .sort_values(['Department', 'Month'])
)

print('train_m built:', train_m.shape)

train_m built: (688, 3)


## 05 - KPI skeleton – join actuals and future months

In [5]:
# === KPI skeleton — history + future months + inventory merge ===
actuals = train_m.rename(columns={'tickets': 'Actual Volume'})[['Department','Month','Actual Volume']]
start_fc = pd.Timestamp.today().to_period('M').to_timestamp()
future_months = pd.date_range(start_fc, periods=HORIZON_MONTHS, freq='MS')

# Keep last 18 months history for context (configurable)
hist_keep = actuals.groupby('Department').tail(18)

kpi = pd.concat(
    [
        hist_keep,
        pd.MultiIndex.from_product(
            [actuals['Department'].unique(), future_months],
            names=['Department','Month']
        ).to_frame(index=False)
    ],
    ignore_index=True
).drop_duplicates(['Department','Month'])

kpi = kpi.merge(actuals, on=['Department','Month'], how='left')

# Merge inventory if present
if not inventory.empty:
    inv_dep  = find_first(inventory, ['Department','department'])
    inv_mon  = find_first(inventory, ['Month','month','Date'])
    inv_val  = find_first(inventory, ['Inventory','inventory','Backlog'])
    if inv_dep and inv_mon and inv_val:
        inv = inventory[[inv_dep, inv_mon, inv_val]].copy()
        inv.columns = ['Department','Month','Inventory']
        inv['Month'] = pd.to_datetime(inv['Month'], errors='coerce').values.astype('datetime64[M]')
        kpi = kpi.merge(inv, on=['Department','Month'], how='left')

## 06 - Forecasts – Prophet and AutoARIMA with exogenous regressors + CV-based hybrid selection

In [6]:
# 6.1 Build exogenous features for all months involved (history + future)
exog_all_months = build_exog_for_months(pd.concat([train_m['Month'], kpi['Month']]))


# 6.2 Load Department -> Vertical mapping (official Excel first, then Einstein)
def load_department_vertical_map_from_df(dept_df: pd.DataFrame) -> dict:
    """Build a {department_name -> vertical} dictionary from a DataFrame."""
    if dept_df is None or len(dept_df) == 0:
        return {}

    def pick(df, candidates):
        low = {c.lower(): c for c in df.columns}
        for cand in candidates:
            if cand.lower() in low:
                return low[cand.lower()]
        # normalized search
        for c in df.columns:
            cl = c.lower().replace(' ', '').replace('_', '')
            for cand in candidates:
                if cand.lower().replace(' ', '').replace('_', '') == cl:
                    return c
        return None

    col_dep  = pick(dept_df, ['department_name', 'Department', 'dept_name'])
    col_vert = pick(dept_df, ['vertical', 'Vertical', 'business unit', 'bu'])

    if not col_dep or not col_vert:
        print("[WARN] department.xlsx missing required columns (department_name/vertical).")
        return {}

    m = (dept_df[[col_dep, col_vert]]
         .dropna().drop_duplicates()
         .rename(columns={col_dep: 'Department_name', col_vert: 'Vertical'}))

    return dict(zip(m['Department_name'], m['Vertical']))


def build_dep_to_vertical_map_from_einstein(einstein_df: pd.DataFrame) -> dict:
    dep_col  = find_first(einstein_df, ['Department','department','Department Name','dept_name'])
    vert_col = find_first(einstein_df, ['Vertical','vertical','Business Unit','BU'])

    if dep_col and vert_col:
        m = (einstein_df[[dep_col, vert_col]]
             .dropna().drop_duplicates()
             .rename(columns={dep_col:'Department', vert_col:'Vertical'}))
        return dict(zip(m['Department'], m['Vertical']))

    return {}


# Build mapping dicts
dep_to_vertical_excel = load_department_vertical_map_from_df(dept_map_df)
dep_to_vertical_ein   = build_dep_to_vertical_map_from_einstein(einstein)

# Merge (Excel is authoritative, overrides Einstein)
dep_to_vertical = {}
dep_to_vertical.update(dep_to_vertical_ein)
dep_to_vertical.update(dep_to_vertical_excel)

print(f"[INFO] Department->Vertical entries from department.xlsx: {len(dep_to_vertical_excel)}")
print(f"[INFO] Department->Vertical entries from einstein: {len(dep_to_vertical_ein)}")


# 6.3 Compute base forecasts (Prophet / ARIMA) using exogenous regressors
TARGET_VERTICALS = {'Payments','Hospitality'}

fc_prophet = forecast_prophet_exog(
    train_m, exog_all_months,
    for_verticals=TARGET_VERTICALS,
    dep_to_vertical=dep_to_vertical,
    horizon=HORIZON_MONTHS
)

fc_arima = forecast_arima_exog(
    train_m, exog_all_months,
    for_verticals=TARGET_VERTICALS,
    dep_to_vertical=dep_to_vertical,
    horizon=HORIZON_MONTHS
)


kpi = (
    kpi
    .merge(fc_prophet, on=['Department','Month'], how='left')
    .merge(fc_arima,   on=['Department','Month'], how='left')
)

for c in ['Forecast_Prophet','Forecast_ARIMA']:
    if c in kpi.columns:
        kpi[c] = kpi[c].fillna(0).astype(int)


# 6.4 CV-based hybrid selection (honest accuracy)
mape_table_cv = build_hybrid_table_cv(
    train_m, exog_all_months,
    for_verticals=TARGET_VERTICALS,
    dep_to_vertical=dep_to_vertical
)

kpi = apply_hybrid(kpi, mape_table_cv, dep_col='Department')


# Optional blending when MAPEs are almost equal (< 1 pp)
if 'MAPE_Prophet_CV' in mape_table_cv.columns and 'MAPE_ARIMA_CV' in mape_table_cv.columns:
    best_map = mape_table_cv.set_index('Department')[
        ['MAPE_Prophet_CV','MAPE_ARIMA_CV','Best_Model']
    ].to_dict('index')

    for dep, info in best_map.items():
        mpp, maa = info.get('MAPE_Prophet_CV'), info.get('MAPE_ARIMA_CV')
        if pd.notna(mpp) and pd.notna(maa) and abs(mpp - maa) < 1.0:
            wp = 1.0 / max(mpp, 1e-6)
            wa = 1.0 / max(maa, 1e-6)
            wsum = wp + wa
            mask = kpi['Department'] == dep
            blend = np.round(
                (wp/wsum) * kpi.loc[mask,'Forecast_Prophet'] +
                (wa/wsum) * kpi.loc[mask,'Forecast_ARIMA']
            ).astype(int)
            kpi.loc[mask, 'Forecast_Hybrid'] = blend.values


kpi['Forecast_Hybrid'] = kpi['Forecast_Hybrid'].fillna(0).astype(int)

[INFO] Department->Vertical entries from department.xlsx: 101
[INFO] Department->Vertical entries from einstein: 101


17:00:32 - cmdstanpy - INFO - Chain [1] start processing
17:00:32 - cmdstanpy - INFO - Chain [1] done processing
17:00:32 - cmdstanpy - INFO - Chain [1] start processing
17:00:32 - cmdstanpy - INFO - Chain [1] done processing
17:00:32 - cmdstanpy - INFO - Chain [1] start processing
17:00:32 - cmdstanpy - INFO - Chain [1] done processing
17:00:32 - cmdstanpy - INFO - Chain [1] start processing
17:00:33 - cmdstanpy - INFO - Chain [1] done processing
17:00:33 - cmdstanpy - INFO - Chain [1] start processing
17:00:33 - cmdstanpy - INFO - Chain [1] done processing
17:00:33 - cmdstanpy - INFO - Chain [1] start processing
17:00:33 - cmdstanpy - INFO - Chain [1] done processing
17:00:33 - cmdstanpy - INFO - Chain [1] start processing
17:00:33 - cmdstanpy - INFO - Chain [1] done processing
17:00:34 - cmdstanpy - INFO - Chain [1] start processing
17:00:34 - cmdstanpy - INFO - Chain [1] done processing
17:00:34 - cmdstanpy - INFO - Chain [1] start processing
17:00:34 - cmdstanpy - INFO - Chain [1]

## 07. Capacity & Productivity enrichment

In [7]:
kpi['Capacity'] = 0
kpi['Productivity'] = 0

# Productivity merge
prod_dep = find_first(productivity, ['Department','department'])
prod_mon = find_first(productivity, ['Month','month','Date'])
prod_val = find_first(productivity, ['Productivity','productivity','Tickets per Agent','TPA'])

if prod_dep and prod_mon and prod_val:
    prod = productivity[[prod_dep, prod_mon, prod_val]].copy()
    prod.columns = ['Department','Month','Productivity']
    prod['Month'] = pd.to_datetime(prod['Month'], errors='coerce').values.astype('datetime64[M]')
    kpi = kpi.drop(columns=['Productivity']).merge(prod, on=['Department','Month'], how='left').fillna({'Productivity':0})

# Capacity merge (agents or call, whichever contains a capacity-like column)
cap_dep = find_first(agents, ['Department','department']) or find_first(call, ['Department','department'])
cap_mon = find_first(agents, ['Month','month','Date']) or find_first(call, ['Month','month','Date'])
cap_val = find_first(agents, ['Capacity','capacity','HC Capacity']) or find_first(call, ['Capacity','capacity'])

if cap_dep and cap_mon and cap_val:
    cap_df = (agents if find_first(agents, ['Capacity','capacity','HC Capacity']) else call)
    cap = cap_df[[cap_dep, cap_mon, cap_val]].copy()
    cap.columns = ['Department','Month','Capacity']
    cap['Month'] = pd.to_datetime(cap['Month'], errors='coerce').values.astype('datetime64[M]')
    kpi = kpi.drop(columns=['Capacity']).merge(cap, on=['Department','Month'], how='left').fillna({'Capacity':0})

# Basic derived gaps (absolute)
kpi['Expected Forecast vs Capacity'] = kpi['Forecast_Hybrid'] - kpi['Capacity']
kpi['Actual Volume vs Productivity'] = kpi.get('Actual Volume', pd.Series(0, index=kpi.index)).fillna(0) - kpi['Productivity']

# If Inventory is missing, approximate as cumulative sum of monthly (Actual - Productivity)
if 'Inventory' not in kpi.columns:
    kpi = kpi.sort_values(['Department','Month'])
    kpi['Inventory'] = kpi.groupby('Department')['Actual Volume vs Productivity'].cumsum()

## 08. Build final report to upgrade to PowerBI 

In [8]:
report = kpi.copy()

# Attach Vertical using the official mapping dictionary
report['Vertical'] = report['Department'].map(dep_to_vertical).fillna('Unknown')

# Coverage logging (helps you fix unmapped departments in department.xlsx)
total_deps  = kpi['Department'].nunique()
mapped_deps = report.loc[report['Vertical']!='Unknown','Department'].nunique()
print(f"[INFO] Vertical mapping coverage: {mapped_deps}/{total_deps} departments mapped.")
if mapped_deps < total_deps:
    missing = sorted(set(kpi['Department']) - set(report.loc[report['Vertical']!='Unknown','Department']))
    print("[WARN] Unmapped departments (sample up to 20):", missing[:20])

# Rename Department to Department_name
report = report.rename(columns={'Department':'Department_name'})

# Core columns
report['Forecast']       = report['Forecast_Hybrid'].astype(float)
report['Actual Volume']  = report.get('Actual Volume', pd.Series(0, index=report.index)).astype(float)

# Forecast Accuracy (as fraction; convert to percent later)
report['Forecast Accuracy'] = np.where(
    report['Actual Volume'] > 0,
    1.0 - (np.abs(report['Forecast'] - report['Actual Volume']) / report['Actual Volume']),
    np.nan
)

# >>> Requested: "Difference Capacity vs Productivity" as PERCENT <<<
# Percentage of productivity over capacity, expressed in percent
report['Difference Capacity vs Productivity'] = np.where(
    report['Capacity'] > 0,
    (report['Productivity'] / report['Capacity']) * 100.0,
    np.nan
)

# Other numeric gaps as requested (keep for context)
report['Expected Forecast vs Capacity'] = report['Forecast'] - report['Capacity']
report['Actual Volume vs Productivity'] = report['Actual Volume'] - report['Productivity']

# Column order EXACTLY as requested
report_out = report[['Vertical',
                     'Department_name',
                     'Month',
                     'Forecast',
                     'Actual Volume',
                     'Forecast Accuracy',
                     'Capacity',
                     'Productivity',
                     'Difference Capacity vs Productivity',
                     'Expected Forecast vs Capacity',
                     'Actual Volume vs Productivity',
                     'Inventory']].copy()

# Friendly formatting
report_out['Month'] = pd.to_datetime(report_out['Month']).dt.strftime('%b-%y').str.title()
report_out['Forecast Accuracy'] = (report_out['Forecast Accuracy'] * 100).round(1)  # %
report_out['Difference Capacity vs Productivity'] = report_out['Difference Capacity vs Productivity'].round(1)

# (Optional) Round some integer-like fields for presentation
for col in ['Forecast','Actual Volume','Capacity','Productivity','Expected Forecast vs Capacity',
            'Actual Volume vs Productivity','Inventory']:
    if col in report_out.columns:
        report_out[col] = pd.to_numeric(report_out[col], errors='coerce').round(0).astype('Int64')

[INFO] Vertical mapping coverage: 0/75 departments mapped.
[WARN] Unmapped departments (sample up to 20): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


## 09. Language split and export to excel

In [9]:
lang_df   = language_split(kpi, fc_col='Forecast_Hybrid')
lang_pivot= lang_df.pivot_table(index=['Department','Month'], columns='Language',
                                values='Forecast_Hybrid_Lang', aggfunc='sum').reset_index()

# Ensure output folder exists
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
out_xlsx = os.path.join(OUTPUT_FOLDER, OUT_XLSX)
out_mape = os.path.join(OUTPUT_FOLDER, OUT_MAPE)

# === Export all sheets ===
with pd.ExcelWriter(out_xlsx, engine=PD_WRITE_XLSX_ENGINE) as writer:
    # Full KPI table for traceability
    kpi.to_excel(writer, sheet_name='kpi_final', index=False)
    # CV table (model selection diagnostics)
    mape_table_cv.to_excel(writer, sheet_name='mape_table_cv', index=False)
    # Language detail
    lang_df.to_excel(writer, sheet_name='forecast_by_language', index=False)
    lang_pivot.to_excel(writer, sheet_name='language_pivot', index=False)
    # Final requested report
    report_out.to_excel(writer, sheet_name='capacity_report', index=False)

# Separate MAPE file (compatibility with your original pipeline)
mape_table_cv.to_excel(out_mape, index=False)

print("Exported:")
print(" -", out_xlsx)
print(" -", out_mape)
print('Rows in KPI:', len(kpi))
print('Departments:', kpi['Department_name'].nunique() if 'Department_name' in kpi.columns else kpi['Department'].nunique())
print('Actuals column present?', 'Actual Volume' in kpi.columns)

Exported:
 - C:\Users\pt3canro\Desktop\CAPACITY\outputs\capacity_forecast_hybrid.xlsx
 - C:\Users\pt3canro\Desktop\CAPACITY\outputs\mape_by_department.xlsx
Rows in KPI: 1102
Departments: 75
Actuals column present? False
